In [1]:
import pandas as pd

In [2]:
senseData = pd.read_csv("Ltrain.csv", header = None)
senseData.columns = ["Case_ID", "Timestamp", "Type", "Sensor"]
senseData

,Case_ID,Timestamp,Type,Sensor
0,2022-09-02,2022-09-02T08:09:31.221024343,PresenceSensor,door_A
1,2022-09-02,2022-09-02T08:11:37.084985372,FloorSensor,sens1_A
2,2022-09-02,2022-09-02T08:14:20.292092153,FloorSensor,sens1_A
3,2022-09-02,2022-09-02T08:14:25.534732839,FloorSensor,sens3_A
4,2022-09-02,2022-09-02T08:14:55.534732839,FloorSensor,sens3_A
...,...,...,...,...
552,2022-09-11,2022-09-11T08:30:27.170242311,FloorSensor,sens1_D
553,2022-09-11,2022-09-11T08:30:57.170242311,FloorSensor,sens1_D
554,2022-09-11,2022-09-11T08:31:27.170242311,FloorSensor,sens1_D
555,2022-09-11,2022-09-11T08:31:57.170242311,FloorSensor,sens1_D


In [3]:
newdataset = pd.DataFrame(columns =["Case_ID", "Timestamp", "Type", "Sensor", "Label", "Activity_ID"])
temp = 0
i = 0

for index, row in senseData.iterrows():
    activity = row["Sensor"][-1]
    new_row = {}
    new_row["Case_ID"] = row["Case_ID"]
    new_row["Timestamp"] = row["Timestamp"]
    new_row["Type"] = row["Type"]
    new_row["Sensor"] = row["Sensor"]
    # First row of the dataset
    if(temp == 0 and i == 0):
        new_row["Label"] = activity
        new_row["Activity_ID"] = i
        temp = activity
    # if the actual row has the same label of the stored, then is the same group
    elif(activity == temp):
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    # if the label mismatch, start a new group
    elif(activity != temp):
        i = i+1
        temp = activity
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    newdataset = newdataset.append(new_row, ignore_index=True)
newdataset
    
    

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02T08:09:31.221024343,PresenceSensor,door_A,A,0
1,2022-09-02,2022-09-02T08:11:37.084985372,FloorSensor,sens1_A,A,0
2,2022-09-02,2022-09-02T08:14:20.292092153,FloorSensor,sens1_A,A,0
3,2022-09-02,2022-09-02T08:14:25.534732839,FloorSensor,sens3_A,A,0
4,2022-09-02,2022-09-02T08:14:55.534732839,FloorSensor,sens3_A,A,0
...,...,...,...,...,...,...
552,2022-09-11,2022-09-11T08:30:27.170242311,FloorSensor,sens1_D,D,39
553,2022-09-11,2022-09-11T08:30:57.170242311,FloorSensor,sens1_D,D,39
554,2022-09-11,2022-09-11T08:31:27.170242311,FloorSensor,sens1_D,D,39
555,2022-09-11,2022-09-11T08:31:57.170242311,FloorSensor,sens1_D,D,39


In [4]:
# Save the file
newdataset.to_csv("Ltrain_labeled.csv", index=False)

In [5]:
columns = ["Timestamp"] + newdataset.Sensor.unique().tolist() + ["Label"]
labeled_senseData = pd.DataFrame(columns=columns)

In [6]:
# Transpose the log and create a row for each activation, incrementing the counter each time
grouped = newdataset.groupby("Activity_ID")
i = 0
for a, group in grouped:
    group = group.reset_index()
    counter = {}
    for ind, el in group.iterrows():
        newrow = {}
        #newrow["Activity_Size"] = len(group)
        newrow["Timestamp"] = el["Timestamp"]
        newrow["Label"] = el["Label"]
        # print(counter)
        if el["Sensor"] in counter:
            counter[el["Sensor"]] = counter[el["Sensor"]]+1
            #newrow[el["Sensor_ID"]] = counter[el["Sensor_ID"]]
        else:
            #newrow[el["Sensor_ID"]] = 1
            counter[el["Sensor"]] =1
        for k, v in counter.items():
            newrow[k] = v
        tempp = pd.DataFrame([newrow])
        # print(newrow)
        labeled_senseData = pd.concat([labeled_senseData, tempp], ignore_index=True)
    i +=1
    # if(i==2):
    #      break
# print(labeled_senseData)
labeled_senseData

,Timestamp,door_A,sens1_A,sens3_A,sens4_A,door_B,sens4_B,sens2_B,sens1_B,sens3_B,...,sens3_C,sens2_C,sens4_C,sens4_D,sens3_D,sens2_D,sens1_D,sens2_A,door_D,Label
0,2022-09-02T08:09:31.221024343,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
1,2022-09-02T08:11:37.084985372,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
2,2022-09-02T08:14:20.292092153,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
3,2022-09-02T08:14:25.534732839,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
4,2022-09-02T08:14:55.534732839,1,2,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,2022-09-11T08:30:27.170242311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,3,1,NaN,NaN,D
553,2022-09-11T08:30:57.170242311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,3,2,NaN,NaN,D
554,2022-09-11T08:31:27.170242311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,3,3,NaN,NaN,D
555,2022-09-11T08:31:57.170242311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,3,4,NaN,NaN,D


In [7]:
# Save the file
labeled_senseData.to_csv("Ltrain_labeled_trans.csv", index=False)